<center><img src="trainers.jpg" alt="Trainers in a store" width=550></center>

Sports clothing and athleisure attire is a huge industry, worth approximately $193 billion in 2021 with a strong growth forecast over the next decade! (`Source: https://www.statista.com/statistics/254489/total-revenue-of-the-global-sports-apparel-market/`)

In this workbook, you will undertake the role of a product analyst for an online sports clothing company. The company is specifically interested in how it can improve revenue. You will dive into product data such as pricing, reviews, descriptions, and ratings, as well as revenue and website traffic, to produce recommendations for its marketing and sales teams.  

### The data:
You've been provided with four datasets to investigate:

`brands.csv`

| Columns | Description |
|---------|-------------|
| `product_id` | Unique product identifier |
| `brand` | Brand of the product | 

`finance.csv`

| Columns | Description |
|---------|-------------|
| `product_id` | Unique product identifier |
| `listing_price` | Original price of the product | 
| `sale_price` | Discounted price of the product |
| `discount` | Discount off the listing price, as a decimal | 
| `revenue` | Revenue generated by the product |

`info.csv`

| Columns | Description |
|---------|-------------|
| `product_name` | Name of the product | 
| `product_id` | Unique product identifier |
| `description` | Description of the product |

`reviews.csv`

| Columns | Description |
|---------|-------------|
| `product_id` | Unique product identifier |
| `rating` | Average product rating | 
| `reviews` | Number of reviews for the product |

In [145]:
# Importing libraries
import pandas as pd

# Loading the data
brands = pd.read_csv("brands.csv") 
finance = pd.read_csv("finance.csv")
info = pd.read_csv("info.csv")
reviews = pd.read_csv("reviews.csv")

# Start coding here...

In [146]:
brands.head()

,product_id,brand
0,AH2430,NaN
1,G27341,Adidas
2,CM0081,Adidas
3,B44832,Adidas
4,D98205,Adidas


In [147]:
finance.head()

,product_id,listing_price,sale_price,discount,revenue
0,AH2430,NaN,NaN,NaN,NaN
1,G27341,75.99,37.99,0.5,1641.17
2,CM0081,9.99,5.99,0.4,398.93
3,B44832,69.99,34.99,0.5,2204.37
4,D98205,79.99,39.99,0.5,5182.70


In [148]:
info.head()

,product_name,product_id,description
0,NaN,AH2430,NaN
1,Women's adidas Originals Sleek Shoes,G27341,"A modern take on adidas sport heritage, tailor..."
2,Women's adidas Swim Puka Slippers,CM0081,These adidas Puka slippers for women's come wi...
3,Women's adidas Sport Inspired Questar Ride Shoes,B44832,"Inspired by modern tech runners, these women's..."
4,Women's adidas Originals Taekwondo Shoes,D98205,This design is inspired by vintage Taekwondo s...


In [149]:
reviews.head()

,product_id,rating,reviews
0,AH2430,NaN,NaN
1,G27341,3.3,24.0
2,CM0081,2.6,37.0
3,B44832,4.1,35.0
4,D98205,3.5,72.0


In [150]:
#brands, finance, info, reviews -> product_id
join1 = pd.merge(left=brands, right=finance, how="left",  on=["product_id"], indicator=True)
join2 = pd.merge(left=join1, right=info, how="left",  on=["product_id"])
join3 = pd.merge(left=join2, right=reviews, how="left",  on=["product_id"])

join3 = join3.drop(columns=['_merge'])
join3.head()

,product_id,brand,listing_price,sale_price,discount,revenue,product_name,description,rating,reviews
0,AH2430,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,G27341,Adidas,75.99,37.99,0.5,1641.17,Women's adidas Originals Sleek Shoes,"A modern take on adidas sport heritage, tailor...",3.3,24.0
2,CM0081,Adidas,9.99,5.99,0.4,398.93,Women's adidas Swim Puka Slippers,These adidas Puka slippers for women's come wi...,2.6,37.0
3,B44832,Adidas,69.99,34.99,0.5,2204.37,Women's adidas Sport Inspired Questar Ride Shoes,"Inspired by modern tech runners, these women's...",4.1,35.0
4,D98205,Adidas,79.99,39.99,0.5,5182.70,Women's adidas Originals Taekwondo Shoes,This design is inspired by vintage Taekwondo s...,3.5,72.0


In [151]:
print(join3.isnull().sum())

product_id        0
brand            59
listing_price    59
sale_price       59
discount         59
revenue          59
product_name     59
description      62
rating           59
reviews          59
dtype: int64


In [152]:
data = join3.dropna(axis=0)
#print(data.isnull().sum())
data.head()

,product_id,brand,listing_price,sale_price,discount,revenue,product_name,description,rating,reviews
1,G27341,Adidas,75.99,37.99,0.5,1641.17,Women's adidas Originals Sleek Shoes,"A modern take on adidas sport heritage, tailor...",3.3,24.0
2,CM0081,Adidas,9.99,5.99,0.4,398.93,Women's adidas Swim Puka Slippers,These adidas Puka slippers for women's come wi...,2.6,37.0
3,B44832,Adidas,69.99,34.99,0.5,2204.37,Women's adidas Sport Inspired Questar Ride Shoes,"Inspired by modern tech runners, these women's...",4.1,35.0
4,D98205,Adidas,79.99,39.99,0.5,5182.70,Women's adidas Originals Taekwondo Shoes,This design is inspired by vintage Taekwondo s...,3.5,72.0
5,B75586,Adidas,47.99,19.20,0.6,1555.20,Women's adidas Sport Inspired Duramo Lite 2.0 ...,Refine your interval training in these women's...,1.0,45.0


In [153]:
labels = ['Budget','Average','Expensive','Elite']

quartile25 = data["listing_price"].quantile(0.25)
median = data["listing_price"].median()
quartile75 = data["listing_price"].quantile(0.75)
max = data["listing_price"].max()

bins = [0,quartile25, median, quartile75, max]
#data["price_label"]= pd.cut(data["listing_price"], labels=labels, bins=bins)
data["price_label"] = pd.qcut(data["listing_price"], q=4, labels=labels)

data = data[data["brand"].isin(['Adidas','Nike'])]
data = data.groupby(["brand","price_label"]).agg({"revenue":["mean","count"]})
data.columns = ["mean_revenue", "num_products"]
data = data.reset_index()
data = data[["brand", "price_label", "num_products", "mean_revenue"]]
data[["num_products", "mean_revenue"]] = data[["num_products", "mean_revenue"]].round(2)
adidas_vs_nike= data
adidas_vs_nike.head()


,brand,price_label,num_products,mean_revenue
0,Adidas,Budget,574,2015.68
1,Adidas,Average,655,3035.30
2,Adidas,Expensive,759,4621.56
3,Adidas,Elite,587,8302.78
4,Nike,Budget,357,1596.33


In [154]:
adidas_vs_nike

,brand,price_label,num_products,mean_revenue
0,Adidas,Budget,574,2015.68
1,Adidas,Average,655,3035.30
2,Adidas,Expensive,759,4621.56
3,Adidas,Elite,587,8302.78
4,Nike,Budget,357,1596.33
5,Nike,Average,8,675.59
6,Nike,Expensive,47,500.56
7,Nike,Elite,130,1367.45


In [155]:
data = join3.dropna(axis=0)
data.head(20)

,product_id,brand,listing_price,sale_price,discount,revenue,product_name,description,rating,reviews
1,G27341,Adidas,75.99,37.99,0.5,1641.17,Women's adidas Originals Sleek Shoes,"A modern take on adidas sport heritage, tailor...",3.3,24.0
2,CM0081,Adidas,9.99,5.99,0.4,398.93,Women's adidas Swim Puka Slippers,These adidas Puka slippers for women's come wi...,2.6,37.0
3,B44832,Adidas,69.99,34.99,0.5,2204.37,Women's adidas Sport Inspired Questar Ride Shoes,"Inspired by modern tech runners, these women's...",4.1,35.0
4,D98205,Adidas,79.99,39.99,0.5,5182.70,Women's adidas Originals Taekwondo Shoes,This design is inspired by vintage Taekwondo s...,3.5,72.0
5,B75586,Adidas,47.99,19.20,0.6,1555.20,Women's adidas Sport Inspired Duramo Lite 2.0 ...,Refine your interval training in these women's...,1.0,45.0
6,CG4051,Adidas,47.99,23.99,0.5,86.36,Women's adidas Sport Inspired Duramo Lite 2.0 ...,Refine your interval training in these women's...,4.4,2.0
7,CM0080,Adidas,9.99,5.99,0.4,75.47,Women's adidas Swim Puka Slippers,These adidas Puka slippers for women's come wi...,2.8,7.0
8,B75990,Adidas,55.99,27.99,0.5,806.11,WOMEN'S ADIDAS RUNNING DURAMO 9 SHOES,These women's neutral running shoes will get y...,4.5,16.0
9,EE5761,Adidas,65.99,39.59,0.4,2779.22,Men's adidas Originals Forest Grove Shoes,The Forest Grove brings back the look of the a...,4.0,39.0
10,EE4553,Adidas,75.99,45.59,0.4,2954.23,Women's adidas Originals Swift Run Shoes,The Swift distills decades of adidas heritage ...,2.7,36.0


In [156]:
import pandas as pd

data["desc_length"] = data["description"].str.len()
#data['desc_length'].max()
bins = [0,100,200,300,400,500,600,700]
labels = ['100', '200', '300', '400', '500', '600', '700']
data["desc_bin"] = pd.cut(data["desc_length"], bins=bins, labels= labels)

data = data.groupby(["desc_bin"]).agg({"reviews": ["sum"], "rating": ["mean"]})
data.columns = ["total_reviews", "mean_rating"]
data = data.reset_index()
data = data.rename(columns={"desc_bin": "description_length"})

# Convert the Interval objects to strings
data["description_length"] = data["description_length"].astype(str)

description_lengths = data[["description_length", "mean_rating", "total_reviews"]]
description_lengths[["mean_rating"]] = description_lengths[["mean_rating"]].round(2)
description_lengths.head(10)

,description_length,mean_rating,total_reviews
0,100,2.26,36.0
1,200,3.19,17719.0
2,300,3.28,76115.0
3,400,3.29,28994.0
4,500,3.35,4984.0
5,600,3.12,852.0
6,700,3.65,818.0
